In [1]:
import time
import numpy as np
import pandas as pd
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from scipy.interpolate import griddata
from sklearn.metrics import mean_squared_error
import tracemalloc  # For memory usage
import subprocess
import numpy as np
import random
from torch.utils.data import DataLoader
from ivyspt.input_processing import split_surfaces, IVSurfaceDataset

# Set the random seed for reproducibility
RANDOM_STATE = 0
N_JOBS = 8
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

In [2]:
hyperparameters = {
    'Input Preprocessing' : {
        'Mask Proportions' : [0.1, 0.3, 0.5, 0.7],
        'Number of Query Points' : None,
        'Batch Size' : 1
    },
}

In [3]:
pre_train_data = pd.read_csv('data/pre_train_data.csv', parse_dates=True, index_col=[0, 1], date_format="ISO8601")
fine_tune_data = pd.read_csv('data/fine_tune_data.csv', parse_dates=True, index_col=[0, 1], date_format="ISO8601")
pre_train_surfaces_train, pre_train_surfaces_validation, pre_train_surfaces_test = split_surfaces(
    pre_train_data,
    toy_sample=True,
    max_points=50,
    max_surfaces=50,
    random_state=RANDOM_STATE
)
fine_tune_surfaces_train, fine_tune_surfaces_validation, fine_tune_surfaces_test = split_surfaces(
    fine_tune_data,
    toy_sample=True,
    max_points=50,
    max_surfaces=50,
    random_state=RANDOM_STATE
)

In [4]:
pre_train_dataset_test = IVSurfaceDataset(
    pre_train_surfaces_test, 
    hyperparameters['Input Preprocessing']['Mask Proportions'], 
    RANDOM_STATE, 
    hyperparameters['Input Preprocessing']['Number of Query Points'] 
)
pre_train_data_loader_test = DataLoader(
    pre_train_dataset_test, 
    batch_size=hyperparameters['Input Preprocessing']['Batch Size'], 
    shuffle=True, 
    num_workers=0, 
    collate_fn=IVSurfaceDataset.collate_fn
)
fine_tune_dataset_test = IVSurfaceDataset(
    fine_tune_surfaces_test, 
    hyperparameters['Input Preprocessing']['Mask Proportions'], 
    RANDOM_STATE, 
    hyperparameters['Input Preprocessing']['Number of Query Points'] 
)
fine_tune_data_loader_test = DataLoader(
    fine_tune_dataset_test, 
    batch_size=hyperparameters['Input Preprocessing']['Batch Size'], 
    shuffle=True, 
    num_workers=0, 
    collate_fn=IVSurfaceDataset.collate_fn
)

In [5]:
import subprocess
import torch

def get_cpu_info():
    # Run the lscpu command
    result = subprocess.run(['lscpu'], stdout=subprocess.PIPE)
    # Decode the output from bytes to string
    lscpu_output = result.stdout.decode('utf-8')
    
    # Parse the lscpu output
    cpu_info = {}
    for line in lscpu_output.split('\n'):
        if line.strip():
            parts = line.split(':', 1)
            if len(parts) == 2:
                key, value = parts
                cpu_info[key.strip()] = value.strip()

    # Extract useful information
    useful_info = {
        "Model name": cpu_info.get("Model name"),
    }

    return useful_info

def format_cpu_info(cpu_info):
    report = (
        f"Model Name: {cpu_info['Model name']}\n"
    )
    return report

def print_device_info():
    # Print CPU info
    cpu_info = get_cpu_info()
    print("CPU Information:")
    print(format_cpu_info(cpu_info))

    # Check if CUDA (GPU) is available
    if torch.cuda.is_available():
        print("\nGPU Information:")
        print(f"Number of GPUs available: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("\nNo GPU available. Running on CPU.")

# Call the function
print_device_info()

CPU Information:
Model Name: Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz


GPU Information:
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 2060 SUPER


In [6]:
def benchmark_models(
    data_loader, 
    model_type='mlp',
    random_state=0
):
    mse_list = []
    total_time = 0
    max_memory_usage = 0

    for batch in data_loader:
        # Extract the data from the batch
        input_surface = batch['Input Surface']
        query_points = batch['Query Points']

        X_train = np.column_stack((
            input_surface['Log Moneyness'][0].numpy(), 
            input_surface['Time to Maturity'][0].numpy()
        ))
        y_train = input_surface['Total Variance'][0].numpy()

        X_test = np.column_stack((
            query_points['Log Moneyness'][0].detach().clone().numpy(), 
            query_points['Time to Maturity'][0].detach().clone().numpy()
        ))
        y_test = query_points['Total Variance'][0].numpy()

        # Start memory and time tracking
        tracemalloc.start()
        start_time = time.time()

        if model_type == 'mlp':
            model = MLPRegressor(max_iter=500, random_state=random_state)

        elif model_type == 'gpr':
            model = GaussianProcessRegressor(random_state=random_state)

        elif model_type == 'quadratic':
            poly = PolynomialFeatures(degree=2)
            X_train_poly = poly.fit_transform(X_train)
            model = LinearRegression()
            model.fit(X_train_poly, y_train)
            X_test_poly = poly.transform(X_test)
            y_pred = model.predict(X_test_poly)

        elif model_type == 'cubic_spline':
            y_pred = griddata(X_train, y_train, X_test, method='cubic')    

        else:
            raise ValueError("Unsupported model type.")

        if model_type in ['mlp', 'gpr']:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

        # Calculate the MSE
        if not np.isnan(y_pred).any():
            mse = mean_squared_error(y_test, y_pred)
            mse_list.append(mse)

        # End memory and time tracking
        current_memory, peak_memory = tracemalloc.get_traced_memory()
        elapsed_time = time.time() - start_time

        tracemalloc.stop()

        total_time += elapsed_time
        max_memory_usage = max(max_memory_usage, peak_memory)

    # Aggregate results
    avg_mse = np.mean(mse_list)
    print(f"Model: {model_type.upper()}")
    print(f"Average MSE: {avg_mse:.6f}")
    print(f"Total Computation Time: {total_time:.2f} seconds")
    print(f"Max Memory Usage: {max_memory_usage / 1024:.2f} KB")

    return avg_mse, total_time, max_memory_usage

In [7]:
# MLP
benchmark_models(pre_train_data_loader_test, model_type='mlp')

Model: MLP
Average MSE: 0.006071
Total Computation Time: 0.75 seconds
Max Memory Usage: 71.62 KB


(0.006071134, 0.7473068237304688, 73334)

In [8]:
# Gaussian Process Regression
benchmark_models(pre_train_data_loader_test, model_type='gpr')

Model: GPR
Average MSE: 59.368100
Total Computation Time: 0.19 seconds
Max Memory Usage: 62.33 KB


(59.36810031723222, 0.18669891357421875, 63822)

In [9]:
# Quadratic Regression
benchmark_models(pre_train_data_loader_test, model_type='quadratic')

Model: QUADRATIC
Average MSE: 0.005918
Total Computation Time: 0.12 seconds
Max Memory Usage: 21.42 KB


(0.0059176637, 0.11544299125671387, 21938)

In [10]:
# Cubic Spline
benchmark_models(pre_train_data_loader_test, model_type='cubic_spline')

Model: CUBIC_SPLINE
Average MSE: nan
Total Computation Time: 0.03 seconds
Max Memory Usage: 22.59 KB


/home/hermes/anaconda3/envs/Apache/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hermes/anaconda3/envs/Apache/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(nan, 0.025411128997802734, 23135)

In [11]:
# MLP
benchmark_models(fine_tune_data_loader_test, model_type='mlp')

/home/hermes/anaconda3/envs/Apache/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/hermes/anaconda3/envs/Apache/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/hermes/anaconda3/envs/Apache/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Model: MLP
Average MSE: 0.170223
Total Computation Time: 1.84 seconds
Max Memory Usage: 199.20 KB


(0.17022325, 1.842632532119751, 203984)

In [12]:
# Gaussian Process Regression
benchmark_models(fine_tune_data_loader_test, model_type='gpr')

Model: GPR
Average MSE: 0.394979
Total Computation Time: 0.22 seconds
Max Memory Usage: 44.51 KB


(0.3949791406405574, 0.21772193908691406, 45581)

In [13]:
# Quadratic Regression
benchmark_models(fine_tune_data_loader_test, model_type='quadratic')

Model: QUADRATIC
Average MSE: 0.170043
Total Computation Time: 0.11 seconds
Max Memory Usage: 10.79 KB


(0.17004265, 0.10647821426391602, 11048)

In [14]:
# Cubic Spline
benchmark_models(fine_tune_data_loader_test, model_type='cubic_spline')

Model: CUBIC_SPLINE
Average MSE: 0.364371
Total Computation Time: 0.03 seconds
Max Memory Usage: 15.00 KB


(0.36437131553383834, 0.03224349021911621, 15356)